# Track overlap vs. Fluence

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cr39py.etch.tools import overlap_fraction
from cr39py.etch.track_overlap_mc import MonteCarloTrackOverlap

As the fluence on a piece of CR-39 increases, so does the chance that two or more tracks may overlap. While in principle it is possible to detect partially overlapping tracks and count them correctly, current generation scanning technology typically attempts to discard these tracks. The result is that, as fluence increases, the effective detection efficency of CR-39 begins to decrease.

cr39py includes a simple Monte-Carlo simulation tool, inspired by A.B. Zylstra et al. 2012, to calculate the fraction of overlapped tracks.

In [ ]:
mc = MonteCarloTrackOverlap(framesize=300, border=20, diameters_mean=10, diameters_std=0)
xyd = mc.draw_tracks(300)
mc.plot_tracks(xyd)

The simulation class includes methods to run large numbers of these simulations in parallel, which generally take at least several minutes to complete. However, A.B. Zylstra et al. 2012 shows that the resulting track overlap vs. fluence curves can be made unique if expressed as a function of a dimensionless parameter

$$
\chi = \eta \bar D^2
$$

Where $\eta$ is the fluence in tracks/cm^2 and $\bar D$ is the mean track diameter in the frame. The resulting output curves are bundled with cr39py, and can be interpolated over using the `overlap_fraction` function.

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlabel("$\chi$")
ax.set_ylabel("$F_n$")
ax.set_ylim(0,1)
chi = np.linspace(0,3,num=200)
fluence = chi / (6*1e-4)**2 # Fluence for 6 um OD tracks in tracks/cm^2
ax2 = ax.twiny()
ax2.set_xlim(np.min(fluence), np.max(fluence))
ax2.set_xlabel("Fluence (tracks/cm$^2$, 6 um OD tracks)")

for Fnum in [1,-1, 2,3,4]:
    Fn = overlap_fraction(Fnum,chi=chi)
    match Fnum:
        case -1:
            lbl = '1-$F_1$'
        case 4:
            lbl = '$F_{4+}$'
        case _:
            lbl = f'$F_{Fnum}$'        
    ax.plot(chi,Fn, label=lbl)
    
F1_zylstra = 1 - chi
F2_zylstra = chi*(1-2*chi/3)
ax.plot(chi, F1_zylstra, label='$F_1$ (Zylstra)', color='k', linestyle='dashed')
ax.plot(chi, F2_zylstra, label='$F_2$ (Zylstra)', color='k', linestyle='dotted')
ax.legend(loc='upper right');

As the fluence increases, the fraction of tracks with no overlapping tracks ($F_1$) decreases, while the fraction of tracks with overlaps increases. 

A.B. Zylstra et al. 2012 derives approximate functions for $F_1$ and $F_2$ 

$$
F_1 = 1 - \chi \\
F_2 = \chi(1-2\chi/3)
$$

We see that these models fit well only for relatively low fluence, $\chi < 0.3$

Since only some limited degree of track overlap is generally tolerable, these curves can be used to determine the approximate maximum acceptable fluence for a given track diameter.